In [9]:
import kfp
from datetime import datetime
from kfp.v2 import compiler
from kfp.v2.google.client import AIPlatformClient
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip

project_id = "feature-store-mars21"
region = "us-central1"
pipeline_root_path = "gs://feature-store-mars21/pl-root/telco-churn-custom"

In [10]:
from kfp.v2.google.client import AIPlatformClient

In [11]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [12]:
BUCKET_NAME = pipeline_root_path

In [13]:
gcs_csv_path = "gs://feature-store-mars21/data/telco/Telco-Customer-Churn.csv"

In [14]:
VERSION= "v3"

@kfp.dsl.pipeline(name="custom-training-{}".format(VERSION),
    pipeline_root=pipeline_root_path)
def pipeline(project_id: str):
        
    dataset_create_op = gcc_aip.TabularDatasetCreateOp(
         project=project_id, display_name="churn-pred", gcs_source=gcs_csv_path,
    )
    
    training_op = gcc_aip.CustomPythonPackageTrainingJobRunOp(
        project=project_id,
        display_name="train_churn_prediction_{}".format(VERSION),
        python_package_gcs_uri="gs://feature-store-mars21/churn-model-dist/churn-0.1.tar.gz",
        python_module="trainer.task",
        model_description="Churn prediction model",
        container_uri='eu.gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest', 
        model_serving_container_image_uri='eu.gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest',
        dataset=dataset_create_op.outputs["dataset"],
        model_display_name="myChurnModel{}".format(VERSION),
        base_output_dir="{}/assets-{}".format(BUCKET_NAME, VERSION),
        staging_bucket="{}/staging".format(BUCKET_NAME),
        machine_type= 'n1-standard-4',
        training_fraction_split = 0.8,
        validation_fraction_split = 0.1,
        test_fraction_split = 0.1
    )

    deploy_op = gcc_aip.ModelDeployOp(  
        model=training_op.outputs["model"],
        project=project_id,
        machine_type="n1-standard-4"
    )
    
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='churn_classif_pipeline.json')

api_client = AIPlatformClient(project_id=project_id, region=region)

response = api_clientl.create_run_from_job_spec(
    'churn_classif_pipeline.json',
    pipeline_root=pipeline_root_path,
    parameter_values={
        'project_id': project_id
    })